# `Studio reversibilità catene P e catene Q`

## `Librerie`

In [ ]:
import generazione_catene, metodi_stazionaria, check, utility
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

## `Test Reversibilità`

### Confronto tra mio metodo e `pydctm`
Confrontiamo la correttezza di reversibilità tra il nostro metodo e quello interno alla libreria `pydctm`

In [ ]:
check.test_confronto_reversibilita(n_matrici=100, n=100)

#### Ora confrontiamo i tempi per il controllo della reversibilità e confrontiamo il nostro metodo e quello di `pydctm`

In [ ]:
# Testa nostro metodo su P reversibili
ris_nostro = check.calcola_tempi("nostro", C=1000, n_punti=50, n_matrici=100)

# Testa PyDTMC su P reversibili
ris_pydtmc = check.calcola_tempi("pydtmc", C=100, n_punti=50, n_matrici=100)

In [ ]:
# Grafico
fig, ax = plt.subplots(figsize=(18,10))
ax.plot([r[0] for r in ris_nostro], [r[1] for r in ris_nostro], 'o-', label="Nostro check")
ax.plot([r[0] for r in ris_pydtmc], [r[1] for r in ris_pydtmc], 's-', label="PyDTMC check")
ax.set_xscale('log')
ax.set_yscale('log')

# Mostra solo griglia principale
ax.grid(True, which='major', linestyle='-', linewidth=0.5)
ax.grid(False, which='minor')

# Forza i tick solo sulle decine
ax.yaxis.set_major_locator(ticker.LogLocator(base=10.0, subs=(1.0,), numticks=100))
ax.xaxis.set_major_locator(ticker.LogLocator(base=10.0, subs=(1.0,), numticks=100))

ax.set_xlabel("Dimensione matrice $n$", fontsize=12)
ax.set_ylabel("Tempo medio (secondi)", fontsize=12)
ax.set_title("Confronto velocità verifica reversibilità su matrici P reversibili non banali", fontsize=14)
ax.legend(fontsize=13)


plt.savefig("P_nostro_vs_pydctm.svg", dpi=300, format="svg", bbox_inches="tight")
plt.show()

### Confronto tra matrici P e matrici Q ben condizionate per la uniformizzazione

In [ ]:
# Testa su P reversibili
ris_nostro = check.calcola_tempi("nostro", C=100, n_punti=50, n_matrici=100)

# Testa su Q reversibili "facili"
ris_pydtmc = check.calcola_tempi("nostro", C=100, n_punti=50, n_matrici=100, discrete=False, forzata=True)

In [ ]:
# Grafico
fig, ax = plt.subplots(figsize=(18,10))
ax.plot([r[0] for r in ris_nostro], [r[1] for r in ris_nostro], 'o-', label="Nostro check su P")
ax.plot([r[0] for r in ris_pydtmc], [r[1] for r in ris_pydtmc], 's-', label="Nostro check su Q")
ax.set_xscale('log')
ax.set_yscale('log')

# Mostra solo griglia principale
ax.grid(True, which='major', linestyle='-', linewidth=0.5)
ax.grid(False, which='minor')

# Forza i tick solo sulle decine
ax.yaxis.set_major_locator(ticker.LogLocator(base=10.0, subs=(1.0,), numticks=100))
ax.xaxis.set_major_locator(ticker.LogLocator(base=10.0, subs=(1.0,), numticks=100))

ax.set_xlabel("Dimensione matrice $n$", fontsize=12)
ax.set_ylabel("Tempo medio (secondi)", fontsize=12)
ax.set_title("Confronto velocità verifica reversibilità su matrici P e Q (ben condizionate) reversibili non banali", fontsize=14)
ax.legend(fontsize=13)


plt.savefig("P_Qbene.svg", dpi=300, format="svg", bbox_inches="tight")
plt.show()

### Confronto tra Q ben condizionate e Q generate senza accortezza

In [ ]:
# Testa su Q reversibili
ris_nostro = check.calcola_tempi("nostro", C=70, n_punti=50, n_matrici=100, discrete=False)

# Testa su Q reversibili "facili"
ris_pydtmc = check.calcola_tempi("nostro", C=70, n_punti=50, n_matrici=100, discrete=False, forzata=True)

In [ ]:
# Grafico
fig, ax = plt.subplots(figsize=(18,10))
ax.plot([r[0] for r in ris_nostro], [r[1] for r in ris_nostro], 'o-', label="Nostro check su Q generiche")
ax.plot([r[0] for r in ris_pydtmc], [r[1] for r in ris_pydtmc], 's-', label="Nostro check su Q ben condizionate")
ax.set_xscale('log')
ax.set_yscale('log')

# Mostra solo griglia principale
ax.grid(True, which='major', linestyle='-', linewidth=0.5)
ax.grid(False, which='minor')

# Forza i tick solo sulle decine
ax.yaxis.set_major_locator(ticker.LogLocator(base=10.0, subs=(1.0,), numticks=100))
ax.xaxis.set_major_locator(ticker.LogLocator(base=10.0, subs=(1.0,), numticks=100))

ax.set_xlabel("Dimensione matrice $n$", fontsize=12)
ax.set_ylabel("Tempo medio (secondi)", fontsize=12)
ax.set_title("Confronto velocità verifica reversibilità su matrici Q ben condizionate e non", fontsize=14)
ax.legend(fontsize=13)


plt.savefig("Qbene_Qnonbene.svg", dpi=300, format="svg", bbox_inches="tight")
plt.show()